In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import random

In [5]:
INPUT = "data/quotes.csv"
# INPUT = "/kaggle/input/quotes-500k/quotes.csv"

ds = pd.read_csv(INPUT)

ds.head()

ds = ds.drop(columns=["author", "category"], axis=1)

ds = np.array(ds)
ds = ds.T[0].astype(str)

# ds = ds[0:1000]

ds = np.char.lower(ds)

ds = np.array(list(map(lambda x: re.sub("[^a-z0-9\s]+", "", x), ds)))

print(ds.shape)
print(ds[0:3])

(499709,)
['im selfish impatient and a little insecure i make mistakes i am out of control and at times hard to handle but if you cant handle me at my worst then you sure as hell dont deserve me at my best'
 'youve gotta dance like theres nobody watchinglove like youll never be hurtsing like theres nobody listeningand live like its heaven on earth'
 'you know youre in love when you cant fall asleep because reality is finally better than your dreams']


In [6]:
print(ds.shape)

MAX_LENGTH = 75

def length_check(x):
    return len(x) < MAX_LENGTH

ds = np.array(list(filter(length_check, ds)))
print(ds.shape)

ds = ds[0:90000]

(499709,)
(131728,)


In [7]:
max_tokens = 20000
max_len = 400

def letters(input):
    valids = []
    for character in input:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)

fifty_most_common_words = [ "the", "be", "of", "and", "a", "to", "in", "he", "have", "it", "that", "for", "they", "I", "with", "as", "not", "on", "she", "at", "by", "this", "we", "you", "do", "but", "from", "or", "which", "one", "would", "all", "will", "there", "say", "who", "make", "when", "can", "more", "if", "no", "man", "out", "other", "so", "what", "time", "up", "go"]



text = " ".join(ds)

def more_than_once(input):
    if len(input) < 1:
        return False
    return text.count(input) > 7

words = text.split(" ")

words = list(set(words))
vocab = sorted(words)

vocab = list(map(letters, vocab))
vocab = list(filter(lambda x: len(x) > 0, vocab))
vocab = list(set(vocab))
vocab = sorted(vocab)

vocab = list(filter(more_than_once, vocab))

print(len(vocab))
print(vocab[0:100])
print(vocab[-10:])


vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_len,
    vocabulary=vocab,
)

vocab = vectorize_layer.get_vocabulary()
vocab[0] = "[UNK]"
vocab[1] = ""
vocab_size = len(vocab)

print(vocab_size)
print(vocab[0:10])



7961
['a', 'aa', 'ab', 'abandon', 'abandoned', 'abba', 'aber', 'abhor', 'abide', 'abilities', 'ability', 'able', 'abled', 'abou', 'abound', 'abounds', 'abour', 'about', 'above', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'abstract', 'absurd', 'absurdity', 'abundance', 'abundant', 'abuse', 'abused', 'abyss', 'ac', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accident', 'accidental', 'accidents', 'accompli', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accord', 'according', 'account', 'accountable', 'accurate', 'accuse', 'accustom', 'accustomed', 'ace', 'aces', 'ache', 'ached', 'aches', 'achieve', 'achieved', 'achievement', 'achievements', 'achiever', 'achieving', 'aching', 'ack', 'acknowledge', 'acquaint', 'acquaintance', 'acquire', 'acquired', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'active', 'activities', 'activity', 'actor', 'actors', 'acts', 'actual', 'actually', 'ad', 'ada', 'adalah', 'ada

In [8]:
lookup = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None, invert=True)

def undo_vectorize(text):
    return lookup(text)

In [ ]:
def pad(x):
    return tf.keras.preprocessing.sequence.pad_sequences(x, padding="post", truncating="post", maxlen=MAX_LENGTH)

def split_input_sequence(x):
    input_text = x[:-1]
    target_text = x[1:]
    return input_text, target_text

dataset = vectorize_layer(ds)

dataset = pad(dataset)

dataset = tf.data.Dataset.from_tensor_slices(dataset)

dataset = dataset.map(split_input_sequence)

BUFFER_SIZE = 1000
BATCH_SIZE = 128

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

for x in dataset.take(1):
    print(x)
    print(np.array(x).shape)

In [ ]:
class QuotesModel(tf.keras.Model):
    def __init__(self, embedding_dim, rnn_units):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        self.rnn = tf.keras.layers.SimpleRNN(rnn_units, return_sequences=True, return_state=True)
        
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.rnn.get_initial_state(x)
        x, states = self.rnn(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        return x

embedding_dim = 256
rnn_units = 2048

model = QuotesModel(embedding_dim, rnn_units)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
for x in dataset.take(1):
#     x = np.array(x[0][0])
#     print(x.shape)
#     y = model.predict(x)
#     print(y.shape)
    print(np.array(x).shape)   
    inputs = np.array(x[0])
    print("Inputs: ", inputs.shape)
    x = inputs
    x = model.embedding(x)
    print("Embedding: ", x.shape)
    x = model.rnn(x)
    # (64,127,2048)
    x = model.dense(x[0])
    print("Dense: ", x.shape)
    
    y = model.predict(inputs)
    print("Y: ", y.shape)
    

In [ ]:
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS)

In [2]:
# model.save("quotes_model")

model = tf.keras.models.load_model("qgt-1")

model.summary()



Model: "quotes_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  2038528   
                                                                 
 simple_rnn (SimpleRNN)      multiple                  4720640   
                                                                 
 dense (Dense)               multiple                  16316187  
                                                                 
Total params: 23,075,355
Trainable params: 23,075,355
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for x in dataset.take(1):
    x = np.array(x[0])
    print(x.shape)
    y = model.predict(x)
    print(y.shape)
    pred = y[:, -1, :]
    print(pred.shape)

In [10]:


class OneStepModel(tf.keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.chars_from_ids = undo_vectorize
        self.ids_from_chars = vectorize_layer

        # skip_ids = self.ids_from_chars(["[UNK]"])[:, :]
        skip_ids = [[0]]

        sparse_mask = tf.SparseTensor(
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(vectorize_layer.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_ids = self.ids_from_chars(inputs)

        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        predicted_logits = predicted_logits[:, -1, :]
        
        predicted_logits = predicted_logits + self.prediction_mask
        
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)
        
        return predicted_chars, states

one_step_model = OneStepModel(model)


def generate_text(model, start_string, num_generate=1000):
    states = None

    next_char = tf.constant([start_string])

    result = [next_char]

    for n in range(num_generate):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result, separator=" ")
    return result[0].numpy().decode("utf-8")

for i in range(10):
    word = random.choice(vocab)
    print(generate_text(one_step_model, start_string="life", num_generate=15))
    print(" ")






ValueError: in user code:

    File "C:\Users\schaefer\AppData\Local\Temp\ipykernel_11372\1882279554.py", line 22, in generate_one_step  *
        predicted_logits, states = self.model(
    File "C:\Users\schaefer\AppData\Roaming\Python\Python38\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "quotes_model" (type QuotesModel).
    
    Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (4 total):
        * Tensor("inputs:0", shape=(None, 400), dtype=int64)
        * None
        * True
        * False
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (4 total):
        * TensorSpec(shape=(None, 74), dtype=tf.int32, name='inputs')
        * None
        * False
        * False
      Keyword

<h1>Version 1.0</h1>
131728 data points (less than 75 characters)<br>
7961 vocab size (6 occurrences and up)<br>
10 Epochs
Acc: .89

Noteworthy Quotes: 
"Chess is a game of grace, and life is a game of chess; it is never too late to have grace, now or never"
"The meaning of life may only be to cope with the lies spread by nerves, but cats seem to have it figured out."
"Guilt without glory isn't great, but my only purpose now is to do anything I can to make up for it."
"Cats may be buried, but they are not gone forever. They are somehow still alive and ignited within us, even if we cannot see them."
"Bliss may be more like the eternal life of trees, rather than the blood that flows through us. Not all of us are only awake and in pain."
"Innocence and guilt breathe life into our choices, carefully projecting the will to move forward. In the end, even John may be abandoned."
"The moon brings us to where we need to be, even if it isn't until afterward. Cats may not need leadership, but we do, and it is never too late to start again."
"Humanity may be facing reality now, but the method to their madness is forever a mystery, even to the cats who seem to have it all figured out."
"Bliss is a myth, more like more trees with eternal blood, though not all awake in pain"

